### <center> This notebook uses the created climatologies and baselines to compute the climatological and intraseasonal anomaly

In [58]:
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
import pandas as pd
from glob import glob
import scipy.signal as signal
import tqdm

## <center> S2S

Load datasets

In [3]:
# Load tropical and coastal S2S data
s2s_tropical = xr.open_dataset('data/HOVMOLLERS/S2S_hovmoller_tropical_2000-2022.nc', decode_timedelta=False)
s2s_coast    = xr.open_dataset('data/HOVMOLLERS/S2S_hovmoller_coastsouth_2000-2022.nc', decode_timedelta=False)
 
trueforecasts = s2s_tropical.inittime.drop_duplicates('inittime')
# Join to a single hovmoller
s2s          = xr.concat([s2s_tropical, s2s_coast], 'distance')
# Drop duplicates and interpolate weekly data to daily
s2s          = s2s.drop_duplicates('inittime').resample({'inittime':'d'}).interpolate()

forecastkind = np.isnan(trueforecasts.reindex({'inittime':s2s.inittime.values}))

In [4]:
# Load tropical and coastal S2S baselines and climatologies
s2s_tropical_baseline = xr.open_dataset('data/HOVMOLLERS/INTRASEASONAL_BASELINES/S2S_intraseasonal-baseline_tropical_2000-2022.nc', decode_timedelta=False)
s2s_coast_baseline    = xr.open_dataset('data/HOVMOLLERS/INTRASEASONAL_BASELINES/S2S_intraseasonal-baseline_coastsouth_2000-2022.nc', decode_timedelta=False) 
# Join to a single hovmoller
s2s_baseline          = xr.concat([s2s_tropical_baseline, s2s_coast_baseline], 'distance')
# Drop duplicates and interpolate weekly data to daily
s2s_baseline          = s2s_baseline.drop_duplicates('inittime').resample({'inittime':'d'}).interpolate()

# Load climatology
s2s_tropical_clim     = xr.open_dataset('data/HOVMOLLERS/CLIMATOLOGIES/S2S_climatology_tropical_2000-2020.nc', decode_timedelta=False)
s2s_coast_clim        = xr.open_dataset('data/HOVMOLLERS/CLIMATOLOGIES/S2S_climatology_coastsouth_2000-2020.nc', decode_timedelta=False)
s2s_clim              = xr.concat([s2s_tropical_clim, s2s_coast_clim], 'distance')

In [5]:
# Compute S2S anomalies and intraseasonal anomalies
s2s_anom  = s2s.groupby('inittime.dayofyear')-s2s_clim
s2s_ianom = s2s-s2s_baseline

In [8]:
encoding_opts = {var: dict(zlib=True, complevel=5) for var in s2s_anom.data_vars}
s2s_anom.to_netcdf('data/HOVMOLLERS/CLIMATOLOGICAL_ANOMALIES/S2S_tropical-coastsouth_anomaly.nc', encoding=encoding_opts)
s2s_ianom.to_netcdf('data/HOVMOLLERS/INTRASEASONAL_ANOMALIES/S2S_tropical-coastsouth_isanomaly.nc', encoding=encoding_opts)

---

In [61]:
# name,drange = 'GLORYS','2000-2020'
for name,drange in zip(['GLORYS','SEALEVEL','ERA5'],['2000-2020','1999-2021','2000-2022']):
    print(name,drange)
    tropical    = xr.open_dataset(f'data/HOVMOLLERS/{name}_hovmoller_tropical_{drange}.nc')
    coast       = xr.open_dataset(f'data/HOVMOLLERS/{name}_hovmoller_coastsouth_{drange}.nc')
    hovmoller   = xr.concat([tropical,coast],'distance')

    tropical_baseline = xr.open_dataset(f'data/HOVMOLLERS/INTRASEASONAL_BASELINES/{name}_intraseasonal-baseline_tropical_{drange}.nc')
    coast_baseline    = xr.open_dataset(f'data/HOVMOLLERS/INTRASEASONAL_BASELINES/{name}_intraseasonal-baseline_coastsouth_{drange}.nc')
    baseline          = xr.concat([tropical_baseline, coast_baseline], 'distance')

    tropical_clim = xr.open_dataset(f'data/HOVMOLLERS/CLIMATOLOGIES/{name}_climatology_tropical_2000-2020.nc')
    coast_clim    = xr.open_dataset(f'data/HOVMOLLERS/CLIMATOLOGIES/{name}_climatology_coastsouth_2000-2020.nc')
    clim          = xr.concat([tropical_clim, coast_clim], 'distance')

    anomaly  = hovmoller.groupby('time.dayofyear')-clim
    ianomaly = hovmoller-baseline.reindex({'time':hovmoller.time.values}, method='nearest')

    encoding_opts = {var: dict(zlib=True, complevel=5) for var in anomaly.data_vars}
    anomaly.to_netcdf(f'data/HOVMOLLERS/CLIMATOLOGICAL_ANOMALIES/{name}_tropical-coastsouth_anomaly.nc', encoding=encoding_opts)
    ianomaly.to_netcdf(f'data/HOVMOLLERS/INTRASEASONAL_ANOMALIES/{name}_tropical-coastsouth_isanomaly.nc', encoding=encoding_opts)
    print(anomaly,'\n\n')


GLORYS 2000-2020
<xarray.Dataset>
Dimensions:    (time: 7671, distance: 2154)
Coordinates:
  * time       (time) datetime64[ns] 2000-01-01T12:00:00 ... 2020-12-31T12:00:00
  * distance   (distance) float64 0.0 9.266 18.53 ... 2.154e+04 2.155e+04
    lat        (distance) float64 0.0 0.0 0.0 0.0 ... -39.75 -39.83 -39.92 -40.0
    lon        (distance) float64 140.0 140.1 140.2 140.2 ... 286.6 286.3 286.2
    index      (distance) int64 0 1 2 3 4 5 6 ... 2148 2149 2150 2151 2152 2153
    dayofyear  (time) int64 1 2 3 4 5 6 7 8 ... 359 360 361 362 363 364 365 366
Data variables:
    zos        (time, distance) float64 0.06261 0.06237 0.06197 ... nan nan nan
    T300       (time, distance) float64 0.7776 0.7764 0.7782 ... nan nan nan
    sst        (time, distance) float64 0.5213 0.5254 0.5306 ... nan nan nan 


SEALEVEL 1999-2021
<xarray.Dataset>
Dimensions:    (time: 8401, distance: 718)
Coordinates:
  * time       (time) datetime64[ns] 1999-01-01 1999-01-02 ... 2021-12-31
  * distance  